# 🧪 Classification & Clustering Lab
## KNN, SVM, K-Means and Evaluation Metrics

---

**Course**: Machine Learning & AI — LUISS Guido Carli

**Objective**: Learn to implement and compare supervised classifiers (KNN, SVM) and unsupervised clustering (K-Means), and evaluate them using appropriate metrics.

**Dataset**: [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/iris) — 150 samples of iris flowers, 4 numerical features, 3 species.

| Feature | Description |
|---------|-------------|
| `sepal length (cm)` | Length of the sepal |
| `sepal width (cm)` | Width of the sepal |
| `petal length (cm)` | Length of the petal |
| `petal width (cm)` | Width of the petal |
| `species` | Target: setosa (0), versicolor (1), virginica (2) |

**What you will learn**:
- How K-Nearest Neighbors works and how to tune K
- How Support Vector Machines find optimal decision boundaries
- How K-Means discovers clusters without labels
- How to evaluate models with precision, recall, F1-score, confusion matrix, silhouette score

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/800px-Iris_versicolor_3.jpg" width="350" alt="Iris flower">

*Image: Iris versicolor — one of the three species in the dataset (Wikimedia Commons)*

---

## 📋 Instructions

- Complete each **Task** by writing code in the empty cells below
- Read the theory sections carefully before attempting each task
- Run cells in order — later tasks depend on earlier results
- If you are stuck, refer to the scikit-learn documentation: [https://scikit-learn.org](https://scikit-learn.org/stable/)
- At the end, you will compare all models and discuss the results

---

## 📦 Part 0: Setup and Data Loading

### Theory: The Iris Dataset

The **Iris dataset** is one of the most famous datasets in machine learning, introduced by statistician **Ronald Fisher** in 1936.

It contains measurements of **150 iris flowers** from three species:

| Species | Samples | Separability |
|---------|---------|--------------|
| Setosa | 50 | Easily separable from the others |
| Versicolor | 50 | Overlaps with Virginica |
| Virginica | 50 | Overlaps with Versicolor |

Each sample has **4 numerical features**: sepal length, sepal width, petal length, and petal width (all in centimeters).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/440px-Kosaciec_szczecinkowaty_Iris_setosa.jpg" width="220" style="display:inline"> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/440px-Iris_versicolor_3.jpg" width="220" style="display:inline"> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/440px-Iris_virginica.jpg" width="220" style="display:inline">

*Left to right: Iris setosa, Iris versicolor, Iris virginica (Wikimedia Commons)*

### ✏️ Task 0.1: Import Libraries and Load Data

Import the following libraries and load the Iris dataset:
- `numpy`, `pandas`, `matplotlib.pyplot`, `seaborn`
- `load_iris` from `sklearn.datasets`

Create a DataFrame with the iris features and add columns for the numeric species label and the species name.

Print the **shape**, the **feature names**, the **class names**, and the **count of samples per class**.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 0.2: Visualize the Data

Create a figure with **two scatter plots side by side**:

1. **Left plot**: Petal length vs petal width, colored by species
2. **Right plot**: Sepal length vs sepal width, colored by species

Add axis labels, a title, and a legend to each plot.

> **Question**: Which pair of features separates the classes better? Why?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 0.3: Pairplot

Create a `seaborn.pairplot` of all 4 features, colored by species name (`hue='species_name'`).

> This visualization shows every 2D combination of features at once. It is a powerful tool for understanding multivariate structure.

In [ ]:
# YOUR CODE HERE

### Theory: Train/Test Split and Feature Scaling

Before training any model, we must:

**1. Split the data** into training and test sets:
- We train on one portion and evaluate on another → honest estimate of generalization
- Common splits: 70/30 or 80/20
- Use **stratified splitting** to maintain class proportions

**2. Scale the features** using `StandardScaler`:

$$z = \frac{x - \mu}{\sigma}$$

This transforms each feature to have **mean = 0** and **std = 1**.

> ⚠️ **Why scale?** Both KNN and SVM are **distance-based** algorithms. Without scaling, features with larger ranges dominate the distance computation.

> ⚠️ **Important**: Always `fit` the scaler on the **training data only**, then `transform` both training and test data. This prevents **data leakage**.

### ✏️ Task 0.4: Split and Scale the Data

1. Separate features (`X`) and target (`y`) from the DataFrame
2. Split into **70% training** and **30% test**, with `random_state=42` and `stratify=y`
3. Apply `StandardScaler`: **fit on training data**, transform both training and test
4. Print the number of samples in each set
5. Print feature means and stds after scaling to verify they are ≈ 0 and ≈ 1

In [ ]:
# YOUR CODE HERE

### ✏️ Task 0.5: Visualize Scaled vs Unscaled Data

Create a figure with **two box plots side by side**:

1. **Left**: Box plot of the **unscaled** training features
2. **Right**: Box plot of the **scaled** training features

> This visualization should make it clear why scaling matters — the unscaled features have very different ranges.

In [ ]:
# YOUR CODE HERE

---

## 🔵 Part 1: K-Nearest Neighbors (KNN)

### Theory

**K-Nearest Neighbors** is one of the simplest and most intuitive machine learning algorithms. It is a **lazy learner**: it does not build a model during training — it stores the entire training set and computes predictions at test time.

**Algorithm:**

```
1. Choose a value for K (number of neighbors)
2. For a new point, compute the distance to every training point
3. Select the K nearest training points
4. Take a majority vote among those K neighbors
5. Assign the winning class
```

**Distance metric** (Euclidean distance):

$$d(p, q) = \sqrt{\sum_{i=1}^{n}(p_i - q_i)^2}$$

**The role of K — the bias-variance tradeoff:**

| K value | Behavior | Risk |
|---------|----------|------|
| Small K (e.g., 1) | Highly sensitive to individual points | **Overfitting** (high variance) |
| Large K (e.g., 100) | Very smooth boundaries | **Underfitting** (high bias) |
| $K \approx \sqrt{n}$ | Common starting heuristic | Balance — but always experiment |

> ⚠️ Because KNN uses distances, **feature scaling is essential**.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/440px-KnnClassification.svg.png" width="300">

*KNN classification: the green dot is classified based on its neighbors (Wikimedia Commons)*

### ✏️ Task 1.1: Train a KNN Classifier

1. Import `KNeighborsClassifier` from `sklearn.neighbors`
2. Create a KNN model with `n_neighbors=5`
3. Fit it on the **scaled** training data
4. Predict on the **scaled** test data
5. Compute and print the accuracy using `accuracy_score` from `sklearn.metrics`

In [ ]:
# YOUR CODE HERE

### ✏️ Task 1.2: Find the Best K

1. Try all values of K from **1 to 30**
2. For each K, train a KNN model and compute the test accuracy
3. **Plot** accuracy vs K (line plot with markers)
4. Add a horizontal dashed red line at the best accuracy value
5. Print the best K and its accuracy

> **Hint**: Use a `for` loop and store accuracies in a list.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 1.3: Visualize KNN Decision Boundary (2D)

Using only the **petal features** (columns 2 and 3 of the scaled data):

1. Train KNN (K=5) on the 2D petal features
2. Create a **mesh grid** with `np.meshgrid` and step size 0.02
3. Predict the class for every point in the mesh
4. Plot the decision regions using `contourf` and overlay the training points

> This gives a visual intuition of how KNN partitions the feature space.

**Hint for mesh grid:**
```python
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
```

In [ ]:
# YOUR CODE HERE

---

## 🟢 Part 2: Support Vector Machine (SVM)

### Theory

**Support Vector Machines** find the hyperplane that separates classes with the **maximum margin** — the largest possible gap between the decision boundary and the closest training points.

**Key Concepts:**

| Concept | Description |
|---------|-------------|
| **Support Vectors** | Training points closest to the boundary — only these define the model |
| **Margin** | Distance between the boundary and the nearest support vectors |
| **Hyperplane** | The decision surface ($w \cdot x + b = 0$ in the linear case) |
| **Kernel Trick** | Implicitly maps data to higher dimensions for non-linear separation |

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/SVM_margin.png/300px-SVM_margin.png" width="320">

*SVM maximizes the margin between classes. Support vectors are circled. (Wikimedia Commons)*

**Common Kernels:**

| Kernel | Formula | Best for |
|--------|---------|----------|
| **Linear** | $K(x_i, x_j) = x_i \cdot x_j$ | Linearly separable data |
| **RBF** (Gaussian) | $K(x_i, x_j) = \exp(-\gamma \|x_i - x_j\|^2)$ | Non-linear boundaries (most common) |
| **Polynomial** | $K(x_i, x_j) = (\gamma \, x_i \cdot x_j + r)^d$ | Polynomial decision surfaces |

**Key Hyperparameters:**

| Parameter | Low value | High value |
|-----------|-----------|------------|
| **C** (regularization) | Smooth boundary → risk underfitting | Complex boundary → risk overfitting |
| **gamma** (RBF/poly) | Long-range influence → risk underfitting | Short-range influence → risk overfitting |

### ✏️ Task 2.1: Train SVMs with Different Kernels

1. Import `SVC` from `sklearn.svm`
2. Train three SVM models with kernels: `'linear'`, `'rbf'`, `'poly'` (use `random_state=42`)
3. Predict on the test set with each model
4. Print the accuracy of each model

> **Question**: Which kernel performs best? Is the difference large?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 2.2: Visualize Support Vectors

Using only the **petal features** (columns 2 and 3 of the scaled data) for visualization:

1. Train an SVM with `kernel='rbf'` on the 2D petal features
2. Create a scatter plot of the training data colored by class
3. **Highlight the support vectors** with larger, hollow red circles (`facecolors='none'`, `edgecolors='red'`)
4. Print the total number of support vectors and the count per class

> **Hint**: After fitting, access `model.support_vectors_` and `model.n_support_`.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 2.3: Effect of the C Parameter

Explore how the regularization parameter C affects the decision boundary:

1. Use C values: `[0.01, 0.1, 1, 10, 100]`
2. For each C, train an SVM (`kernel='rbf'`) on the 2D petal features
3. Plot the **decision boundary** using `contourf` on a mesh grid
4. Overlay the training points
5. In each subplot title, show: C value, test accuracy, number of support vectors

> Create a figure with **5 subplots in a row** (`1 x 5`), one per C value.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 2.4: Visualize SVM Decision Boundary (2D)

Using the **petal features** and the best-performing kernel:

1. Train SVM on 2D petal features
2. Plot the decision boundary using `contourf` (same mesh technique as Task 1.3)
3. Overlay training points and highlight support vectors

> Compare this boundary visually with the KNN boundary from Task 1.3. Which looks smoother?

In [ ]:
# YOUR CODE HERE

---

## 🟡 Part 3: K-Means Clustering

### Theory

Now we switch from **supervised** to **unsupervised** learning. K-Means partitions data into K clusters **without using labels**.

**Algorithm:**

```
1. INITIALIZE: Place K centroids randomly
2. ASSIGN:     Assign each point to the nearest centroid
3. UPDATE:     Move each centroid to the mean of its assigned points
4. REPEAT:     Steps 2-3 until convergence (centroids stop moving)
```

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/K-means_convergence.gif/440px-K-means_convergence.gif" width="350">

*K-Means convergence animation: centroids move iteratively until stable (Wikimedia Commons)*

**Properties:**
- You must specify **K** in advance
- Sensitive to initialization → use `n_init=10` (run multiple times, keep best)
- Assumes **spherical, equally-sized** clusters
- **Feature scaling** is important (distance-based)

**How to choose K:**

| Method | How it works | What to look for |
|--------|-------------|-----------------|
| **Elbow Method** | Plot inertia (within-cluster sum of squares) vs K | The "elbow" where inertia stops dropping sharply |
| **Silhouette Score** | Measures cluster cohesion vs separation `[-1, 1]` | The K with the highest score |

> ⚠️ Always combine algorithmic criteria with **domain knowledge**.

### ✏️ Task 3.1: Elbow Method

1. Import `KMeans` from `sklearn.cluster`
2. Try K values from **1 to 10**
3. For each K, train K-Means (with `random_state=42`, `n_init=10`) and record `inertia_`
4. **Plot** inertia vs K with markers
5. Annotate the elbow point on the plot

> **Question**: Where is the elbow? Does it match the true number of species?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 3.2: Silhouette Score Analysis

1. Import `silhouette_score` from `sklearn.metrics`
2. For K values from **2 to 10**, compute the silhouette score
3. **Plot** silhouette score vs K
4. Print all scores and identify the best K

> **Question**: Does the silhouette method agree with the elbow method? If not, why?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 3.3: Compare Clusters with True Labels

1. Train K-Means with **K=3** on the scaled training data
2. Create a figure with **two scatter plots side by side** (using petal features):
   - **Left**: True species labels
   - **Right**: K-Means cluster assignments, with centroids plotted as large red **X** markers
3. Create a **cross-tabulation** (`pd.crosstab`) comparing true labels with cluster assignments
4. Print and analyze the table

> **Question**: How well did K-Means recover the true species grouping?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 3.4: Visualize Cluster Silhouette per Sample

Using `sklearn.metrics.silhouette_samples`, create a **silhouette plot** for K=3:

1. Compute per-sample silhouette values
2. For each cluster, plot horizontal bars sorted by silhouette value
3. Add a vertical dashed line at the mean silhouette score
4. Color each cluster differently

> This plot reveals whether all clusters are well-formed or if some contain poorly-assigned points.

**Hint:**
```python
from sklearn.metrics import silhouette_samples
sample_silhouette_values = silhouette_samples(X_train_scaled, cluster_labels)
```

In [ ]:
# YOUR CODE HERE

---

## 📊 Part 4: Evaluation Metrics

### Theory: Classification Metrics (Supervised)

| Metric | Formula | What it answers |
|--------|---------|----------------|
| **Precision** | $\frac{TP}{TP + FP}$ | Of predicted positives, how many are actually positive? |
| **Recall** | $\frac{TP}{TP + FN}$ | Of actual positives, how many did we find? |
| **F1-Score** | $2 \cdot \frac{P \cdot R}{P + R}$ | Harmonic mean — balances precision and recall |
| **Accuracy** | $\frac{TP + TN}{Total}$ | Overall fraction of correct predictions |

**Averaging strategies for multi-class:**
- **Macro**: Average across classes (treats all classes equally)
- **Weighted**: Weighted by support (accounts for imbalance)

**Confusion Matrix**: An $n \times n$ table where rows = true classes, columns = predictions. Diagonal = correct, off-diagonal = errors.

### Theory: Clustering Metrics (Unsupervised)

| Metric | Range | What it measures |
|--------|-------|-----------------|
| **Silhouette Score** | $[-1, 1]$ | How similar a point is to its own cluster vs other clusters |
| **Adjusted Rand Index** | $[-1, 1]$ | Agreement between cluster labels and true labels (1 = perfect) |
| **Inertia** | $[0, \infty)$ | Within-cluster sum of squares (lower = tighter clusters) |

### ✏️ Task 4.1: Classification Reports

1. Import `classification_report` from `sklearn.metrics`
2. Print the **classification report** for KNN (K=5) predictions
3. Print the **classification report** for the best SVM model
4. Use `target_names=iris.target_names` for readable output

> **Question**: Which class has the highest F1-score? Which has the lowest? Why?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 4.2: Confusion Matrices

1. Import `ConfusionMatrixDisplay` from `sklearn.metrics`
2. Create a figure with **2 confusion matrices side by side**:
   - Left: KNN predictions (`cmap='Blues'`)
   - Right: Best SVM predictions (`cmap='Greens'`)
3. Use `display_labels=iris.target_names`

> **Question**: Which species pair causes the most confusion? Is this consistent with the scatter plots?

In [ ]:
# YOUR CODE HERE

### ✏️ Task 4.3: Clustering Evaluation

Evaluate K-Means clustering (K=3):

1. Compute **Silhouette Score** on the training data
2. Compute **Adjusted Rand Index** comparing `y_train` with cluster labels
3. Report **Inertia** from `kmeans.inertia_`
4. Print all three metrics with a brief interpretation of each

In [ ]:
# YOUR CODE HERE

### ✏️ Task 4.4: Precision-Recall Bar Chart

Create a **grouped bar chart** comparing precision and recall for each class, for both KNN and SVM:

1. Extract per-class precision and recall from the classification reports (use `classification_report(..., output_dict=True)`)
2. Plot grouped bars: 3 classes × 2 metrics × 2 models
3. Add a legend and axis labels

> This visualization makes it easier to spot which classes are harder for each model.

In [ ]:
# YOUR CODE HERE

---

## 🏆 Part 5: Model Comparison

### ✏️ Task 5.1: Build a Comparison Table

Create a `pandas.DataFrame` comparing all supervised models with these columns:
- Model name
- Accuracy
- Macro F1-Score

Sort by accuracy (descending) and display the table.

**Models to include**: KNN (K=5), SVM (Linear), SVM (RBF), SVM (Poly)

**Hint**: Use `f1_score(y_test, y_pred, average='macro')` from `sklearn.metrics`.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 5.2: Comparison Bar Chart

Create a **grouped bar chart** comparing accuracy and macro F1-score across all 4 models.

> Visualizing the comparison table makes differences immediately obvious.

In [ ]:
# YOUR CODE HERE

### ✏️ Task 5.3: Discussion

Answer the following questions (in a markdown cell or as print statements):

1. Which algorithm performed best on this dataset? Was the difference significant?
2. In what scenarios would you prefer **KNN** over **SVM**, and vice versa?
3. How well did **K-Means** recover the true clusters without labels?
4. Why is **accuracy alone** not sufficient to evaluate a classifier?
5. What would change if the dataset were **highly imbalanced** (e.g., 90% setosa)?

In [ ]:
# YOUR CODE HERE